<a href="https://colab.research.google.com/github/hwankang/Defense/blob/main/0702lstm_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
import datetime
import pandas as pd
from google.colab import files 
uploaded = files.upload()
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
#데이터 읽어오기
import io
data = pd.read_csv(io.BytesIO(uploaded['005930.KS.csv']),  encoding = 'euc-kr')

data.head()
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
#column 제거 방법 제시하라. 
data1=data.drop(['Adj Close'],axis=1)# axis=1 은 종으로 제거 
data1.head()

In [ ]:
#데이터 타입 확인
data1.dtypes
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
#데이터 전처리(2) Object->Datetime
data1['Date'] = pd.to_datetime(data1['Date'], format='%Y-%m-%d %H:%M:%S')
data1['year'] =data1['Date'].dt.year
data1['month'] =data1['Date'].dt.month
data1['day'] =data1['Date'].dt.day
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
#데이터 전처리(1) 결측치 제거
data1 = data1.dropna()
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
#데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글
plt.figure(figsize=(16, 9))
sns.lineplot(y=data1['Close'], x=data1['Date'])
plt.xlabel('time')
plt.ylabel('price')
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글


In [ ]:
#가격 예측을 위해 mid_prices컬럼 만들기
high_prices = data1['High'].values
low_prices = data1['Low'].values
mid_prices = (high_prices + low_prices) / 2
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
#Window만들기(ppt1)
seq_len = 50
sequence_length = seq_len + 1 #51

result = []
for index in range(len(mid_prices) - sequence_length):
    result.append(mid_prices[index: index + sequence_length])
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
#Window 정규화
normalized_data = []
for window in result:
    normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
    normalized_data.append(normalized_window)

result = np.array(normalized_data)
#[출처] [LSTM] 삼성 주가 예측|작성자 윤동글

In [ ]:
result[1]

In [ ]:
result.shape[0]

In [ ]:
result

In [ ]:
row = int(round(result.shape[0] * 0.9))
row


In [ ]:
train = result[:row, :]  #150 51
np.random.shuffle(train)

In [ ]:
x_train = train[:, :-1]# 150 6
x_train.shape

In [ ]:
x_train = train[:, :-1]# 150 6
x_train.shape
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
y_train = train[:, -1]
y_train.shape

In [ ]:
#x_test에 50개 넣고 y_test에 마지막 1개
x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

In [ ]:
x_train.shape, x_test.shape

In [ ]:
#모델을 순차적으로 정의
model = Sequential()

#LSTM 인풋 50개
model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))

#인풋 64개 이거는 조정할 수 있음
model.add(LSTM(64, return_sequences=False))

#다음날 하루의 가격을 예측
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

#50개가 들어가서 1개가 나온다
model.summary()

In [ ]:
#트레이닝 모델을 학습시킨다.
model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    #한번에 몇개씩 묶어서 학습시킬거냐
    batch_size=10,
    epochs=20)

In [ ]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white', figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='Prediction')
ax.legend()
plt.show()